In [1]:
import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
start_time = time.time()

In [ ]:
preproc.do_preprocessing('../data/twitter-datasets/train_pos.txt')
preproc.do_preprocessing('../data/twitter-datasets/train_neg.txt')

In [3]:
lines, y = preproc.return_processed_trainset_and_y(False)

In [4]:
print(len(lines))
lines[:5]

200000


['i do not know justin read my mention or not . only justin and god knows about that , but i hope you will follow me <hashtag> believe number',
 'because your logic is so dumb , i will not even crop out your name or your photo . tsk .',
 '" just put casper in a box ! " lovedloved the battle ! <hashtag> c rak bitch',
 'thanks sir > > do not trip little mama . just keep doin ya thang !',
 'visiting my brother tmr is the bestest birthday gift everever ! ! !']

In [ ]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,1), sublinear_tf=True, max_features=50)
X = vectorizer.fit_transform(lines)

In [ ]:
elapsed_time = divmod(round((time.time() - start_time)), 60)
print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))

BELOW: testing...

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv1D, Embedding, Activation
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding


import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import helpers
import models
import word2vec as w2v

In [ ]:
lines, y = preproc.return_processed_trainset_and_y(False)
len(lines)

In [ ]:
lines1 = lines[:50000] + lines[100000:150000]
y1 = np.concatenate((y[:50000], y[100000:150000]), axis=0)

In [ ]:
len(lines1)

In [ ]:
len(y1)

In [ ]:
adam = Adam(lr=0.1, beta_1=0.9, epsilon=None, decay=0.1, amsgrad=False)

In [ ]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,2), sublinear_tf=True, max_features=200)
X = vectorizer.fit_transform(lines)

In [ ]:
X.shape

In [ ]:
X

In [ ]:
# create model
model = Sequential()
# model.add(Embedding(X.shape[1], 1000, input_length=2000))
# model.add(Conv1D(filters=1, kernel_size=2, padding='valid', activation='relu', strides=1))
model.add(Dense(units=32, input_dim=X.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.add(Activation('softmax'))
model.summary()

In [ ]:
len(lines1)

In [ ]:
#compile model
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
X.shape

In [ ]:
#fit the model
model.fit(x=X, y=y, batch_size=32, epochs=5)

In [ ]:
scores = model.evaluate(X, y1)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# define documents
docs = ['Well done!',
'Good work',
'Great effort',
'nice work',
'Excellent!',
'Weak',
'Poor effort!',
'not good',
'poor work',
'Could have done better.']
# define class labels
labels = ([1,1,1,1,1,0,0,0,0,0])

In [ ]:
from keras.preprocessing.text import one_hot
# integer encode the documents

vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv1D, Embedding, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding

In [ ]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

In [ ]:
# fit the model
model.fit(padded_docs, labels, epochs=100, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))